In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import useful_functions as uf

#load_transform_split(fpath='data/ALL_YEARS_ADDED_FEATURES.csv',
#                         target='DROPOUT_N', expand=False, split=0.1, clean=True,
#                         drop_feats=['SCHOOL_YEAR','DIV_NAME','SCH_NAME','DIPLOMA_RATE'],
#                         fmt='numpy',return_pipeline=False):

X_train_scaled, X_test_scaled, y_train, y_test = uf.load_transform_split(target='DROPOUT_N')

In [2]:
# global variables about whether or not to modify the hyperparameters
USE_BATCH_NORMALIZATION = False
USE_ADAM_OPTIMIZER = True
USE_DROPOUT_REGULARIZATION = False
USE_SPECIAL_INITIALIZATION = True
USE_LR_SCHEDULER = True

# some of the modifications to the hyperparams -- don't really expect these to change
ACTIVATOR = 'selu' # the one suggested by slides 21 & 44 on Lecture 11
INITIALIZER = 'he_normal' # the one suggested by slides 17 & 44 on Lecture 11

# some more modifications to the hyperparams -- probably change these later on
# for now, set them to the default values
DROPOUT_RATE = 0.0
ADAM_LR = 0.001
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
NUM_LAYERS = 10
N_EPOCHS = 30
USE_EARLY_STOPPING = True
LR_10FOLD_DECAY_TIME = 20 # decay by a factor of 10 every X epochs

In [3]:
import tensorflow as tf
from tensorflow import keras

def initialize_model(use_drop=USE_DROPOUT_REGULARIZATION, use_batch=USE_BATCH_NORMALIZATION,
                    drop_rate=DROPOUT_RATE, num_layers=NUM_LAYERS,
                    use_init=USE_SPECIAL_INITIALIZATION, init=INITIALIZER, activ=ACTIVATOR):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train_scaled.shape[1:]))
        
    for i in np.arange(19.0, 1, num_layers/(-18)):
        j = int(i)
        if use_drop:
            model.add(keras.layers.Dropout(rate=drop_rate))
        if use_init:
            model.add(keras.layers.Dense(j, activation=activ, kernel_initializer=init))
        else:
            model.add(keras.layers.Dense(j, activation=activ))
        if use_batch:
            model.add(keras.layers.BatchNormalization())
    if use_init:
        model.add(keras.layers.Dense(1, kernel_initializer=init))
    else:
        model.add(keras.layers.Dense(1))
    return(model)

In [4]:
def compile_model(model, use_adam=USE_ADAM_OPTIMIZER, lr=ADAM_LR, b1=ADAM_BETA1, b2=ADAM_BETA2):
    if use_adam:
        optimizer=keras.optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
    else:
        model.compile(loss='mean_squared_error')
    return(model)

In [16]:
def fit_model(model, use_sched=USE_LR_SCHEDULER, decay_time=LR_10FOLD_DECAY_TIME, init=ADAM_LR,
              use_adam=USE_ADAM_OPTIMIZER, n_epo=N_EPOCHS, stop_early=USE_EARLY_STOPPING):
    if not use_adam or not use_sched:
        history=model.fit(X_train_scaled, y_train, epochs=n_epo)
    else:
        callbacks = []
        if use_sched:
            def lr_fn(epoch): return(init * 10**(-epoch/decay_time))
            callbacks.append(keras.callbacks.LearningRateScheduler(lr_fn))
        if stop_early:
            callbacks.append(keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True))
        history = model.fit(X_train_scaled, y_train, epochs=n_epo, callbacks=callbacks, validation_split=0.2)
    return(history, model)

In [6]:
from sklearn.metrics import mean_squared_error
def analyze_performance(history, model):
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('RMSE: %.2f' %rmse)
    return(rmse)

In [13]:
# can I just do this use *args or something? IDK
def run(use_drop=USE_DROPOUT_REGULARIZATION, use_batch=USE_BATCH_NORMALIZATION,
        drop_rate=DROPOUT_RATE, num_layers=NUM_LAYERS,
        use_init=USE_SPECIAL_INITIALIZATION, init=INITIALIZER, activ=ACTIVATOR,
        use_adam=USE_ADAM_OPTIMIZER, lr=ADAM_LR, b1=ADAM_BETA1, b2=ADAM_BETA2,
        use_sched=USE_LR_SCHEDULER, decay_time=LR_10FOLD_DECAY_TIME, n_epo=N_EPOCHS, stop_early=USE_EARLY_STOPPING):
    
    model = initialize_model(use_drop, use_batch, drop_rate, num_layers, use_init, init, activ)
    model = compile_model(model, use_adam, lr, b1, b2)
    history, model = fit_model(model, use_sched, decay_time, lr, use_adam, n_epo, stop_early)
    rmse = analyze_performance(history, model)
    return(history, model, rmse)

In [17]:
history, model, rmse = run(n_epo=100, decay_time=30)

Train on 51890 samples, validate on 12973 samples
Epoch 1/100
51890/51890 [==============================] - 8s 155us/sample - loss: 4.3669 - val_loss: 5.0416
Epoch 2/100
51890/51890 [==============================] - 6s 125us/sample - loss: 3.6114 - val_loss: 4.6048
Epoch 3/100
51890/51890 [==============================] - 10s 189us/sample - loss: 3.2189 - val_loss: 4.4443
Epoch 4/100
51890/51890 [==============================] - 9s 170us/sample - loss: 3.0196 - val_loss: 4.0984
Epoch 5/100
51890/51890 [==============================] - 15s 286us/sample - loss: 2.8364 - val_loss: 3.8400
Epoch 6/100
51890/51890 [==============================] - 16s 317us/sample - loss: 2.9077 - val_loss: 3.9872
Epoch 7/100
51890/51890 [==============================] - 11s 204us/sample - loss: 3.0461 - val_loss: 4.7686
Epoch 8/100
51890/51890 [==============================] - 8s 149us/sample - loss: 3.1136 - val_loss: 4.1222
Epoch 9/100
51890/51890 [==============================] - 8s 145us/sample

51890/51890 [==============================] - 7s 141us/sample - loss: 2.2120 - val_loss: 3.2290
Epoch 76/100
51890/51890 [==============================] - 8s 149us/sample - loss: 2.2117 - val_loss: 3.2359
Epoch 77/100
51890/51890 [==============================] - 6s 118us/sample - loss: 2.2114 - val_loss: 3.2311
Epoch 78/100
51890/51890 [==============================] - 6s 121us/sample - loss: 2.2104 - val_loss: 3.2291
Epoch 79/100
51890/51890 [==============================] - 7s 127us/sample - loss: 2.2096 - val_loss: 3.2289
Epoch 80/100
51890/51890 [==============================] - 8s 149us/sample - loss: 2.2095 - val_loss: 3.2284
Epoch 81/100
51890/51890 [==============================] - 8s 146us/sample - loss: 2.2094 - val_loss: 3.2276
Epoch 82/100
51890/51890 [==============================] - 7s 135us/sample - loss: 2.2087 - val_loss: 3.2289
RMSE: 1.58


In [ ]:
# learn faster -- the loss was decreasing pretty slowly before
history, model, rmse = run(lr=0.001)

Train on 64863 samples
Epoch 1/30
64863/64863 [==============================] - 157s 2ms/sample - loss: 4.5513
Epoch 2/30
64863/64863 [==============================] - 126s 2ms/sample - loss: 4.5492
Epoch 3/30
64863/64863 [==============================] - 119s 2ms/sample - loss: 4.5433
Epoch 4/30
64863/64863 [==============================] - 120s 2ms/sample - loss: 4.5408
Epoch 5/30
64863/64863 [==============================] - 121s 2ms/sample - loss: 4.5404
Epoch 6/30
64863/64863 [==============================] - 121s 2ms/sample - loss: 4.5415
Epoch 7/30
64863/64863 [==============================] - 121s 2ms/sample - loss: 4.5408
Epoch 8/30
64863/64863 [==============================] - 130s 2ms/sample - loss: 4.5336
Epoch 9/30
64863/64863 [==============================] - 126s 2ms/sample - loss: 4.5368
Epoch 10/30
64863/64863 [==============================] - 121s 2ms/sample - loss: 4.5354
Epoch 11/30
64863/64863 [==============================] - 120s 2ms/sample - loss: 4.5

In [18]:
history.history

{'loss': [4.366924184796254,
  3.61136914084334,
  3.2188835397417273,
  3.019615872964438,
  2.8364456270213854,
  2.9077159593734327,
  3.0460632276250164,
  3.1136287702625487,
  2.888588472767174,
  2.786225790484927,
  2.808483366002281,
  2.8458719423566623,
  3.0144425138110345,
  2.791646230002547,
  2.700002121865623,
  2.678068341352412,
  2.698746129638957,
  2.6502302295051114,
  2.610062224236865,
  2.547221835492594,
  2.4944142985559883,
  2.454124903095368,
  2.4358208498804985,
  2.4121204198681796,
  2.393923916897578,
  2.4132180872990876,
  2.3922308584104086,
  2.3828636112436437,
  2.3658543868799113,
  2.350007328129752,
  2.3513827438241,
  2.35981289684761,
  2.34040990737528,
  2.3306415417815107,
  2.3130823144657255,
  2.309947010464098,
  2.3094986305780827,
  2.309954740459415,
  2.304480988183539,
  2.292706018221082,
  2.293803950269504,
  2.2835321007792166,
  2.27595340010058,
  2.273564491972204,
  2.263337734141178,
  2.259856311460294,
  2.254741813

In [19]:
import pickle
fname = 'good_neural_net'
pickle.dump(model, open(fname, 'wb'))

TypeError: can't pickle _thread.RLock objects

In [20]:
model.save(fname)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


FailedPreconditionError: good_neural_net/variables; Not a directory

In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(19, activation='relu', input_shape=X_train_scaled.shape[1:]))
model.add(keras.layers.BatchNormalization())
for i in range(17, 1, -2):
    model.add(keras.layers.Dense(i, activation='relu'))
    model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1))

model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2)

Train on 51890 samples, validate on 12973 samples
Epoch 1/20
51890/51890 [==============================] - 11s 219us/sample - loss: 3.5021 - val_loss: 3.7694
Epoch 2/20
51890/51890 [==============================] - 9s 177us/sample - loss: 2.8638 - val_loss: 3.3172
Epoch 3/20
51890/51890 [==============================] - 9s 173us/sample - loss: 2.7562 - val_loss: 2.9667
Epoch 4/20
51890/51890 [==============================] - 8s 153us/sample - loss: 2.6725 - val_loss: 3.0613
Epoch 5/20
51890/51890 [==============================] - 8s 160us/sample - loss: 2.6169 - val_loss: 2.9736
Epoch 6/20
51890/51890 [==============================] - 8s 156us/sample - loss: 2.5084 - val_loss: 2.7974
Epoch 7/20
51890/51890 [==============================] - 8s 156us/sample - loss: 2.4934 - val_loss: 2.5694
Epoch 8/20
51890/51890 [==============================] - 8s 155us/sample - loss: 2.4545 - val_loss: 2.5054
Epoch 9/20
51890/51890 [==============================] - 8s 151us/sample - loss: 2.4

In [8]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE: %.3f' %rmse)

RMSE: 1.429


In [9]:
# beta(1,2) correspond to the importance of (momentum, scale(?))
# defaults are lr = 0.001, beta1 = 0.9, beta2 = 0.999
optimizer = keras.optimizers.Adam(learning_rate=ADAM_LR, beta_1=ADAM_BETA1, beta_2=ADAM_BETA2)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# adjust the learning rate
def lr_fn(epoch, init=ADAM_LR, decay_time=LR_10FOLD_DECAY_TIME):
    return(init * 10**(-epoch/decay_time))
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_fn)
history = model.fit(X_train_scaled, y_train, epochs=N_EPOCHS, callbacks=[lr_scheduler])

NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential()
if USE_DROPOUT_REGULARIZATION:
    model.add(keras.layers.Dropout(rate=DROPOUT_RATE, input_shape=X_train_scaled.shape[1:]))
model.add(keras.layers.Dense(19, activation=ACTIVATOR, kernel_initializer=INITIALIZER))
if USE_BATCH_NORMALIZATION:
    model.add(keras.layers.BatchNormalization())
for i in np.arange(19, 1, -18/NUM_NODES): # always start with 19 and end before 1
    j = int(i)
    model.add(keras.layers.Dropout(rate=DROPOUT_RATE))
    model.add(keras.layers.Dense(j, activation=ACTIVATOR, kernel_initializer=INITIALIZER))
    if USE_BATCH_NORMALIZATION:
        model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, kernel_initializer=INITIALIZER)) # the final, predicting layer
model.summary()

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)

In [10]:
keras.__version__

'2.2.4-tf'

In [9]:
tf.__version__

'2.1.0'

In [ ]:
help(keras.optimizers.Adam)

In [ ]:
for i in np.arange(1, 10, 1.5):
    print(i)

In [ ]:
for i in np.arange(19, 1, -18/NUM_NODES):
    print(i)

In [ ]:
NUM_NODES